In [33]:
es_train_data_file = "C:/Users/dksat/Documents/GitHub/50.007-Machine-Learning-Project/ES/train"
es_test_data_file = "C:/Users/dksat/Documents/GitHub/50.007-Machine-Learning-Project/ES/dev.in"
es_predicted_data_file = "C:/Users/dksat/Documents/GitHub/50.007-Machine-Learning-Project/ES/dev.out"
es_actual_data_file = "C:/Users/dksat/Documents/GitHub/50.007-Machine-Learning-Project/ES/dev.out"
ru_train_data_file = "C:/Users/dksat/Documents/GitHub/50.007-Machine-Learning-Project/RU/train"
ru_test_data_file = "C:/Users/dksat/Documents/GitHub/50.007-Machine-Learning-Project/RU/dev.in"
ru_predicted_data_file = "C:/Users/dksat/Documents/GitHub/50.007-Machine-Learning-Project/RU/dev.out"
ru_actual_data_file = "C:/Users/dksat/Documents/GitHub/50.007-Machine-Learning-Project/RU/dev.out"


In [90]:
import itertools
def _preprocess_training_file(training_file):
    tags = []
    tags_with_start_stop = []
    words = []    

    with open(training_file,"r",encoding="utf8") as f:
        document = f.read().rstrip()
        lines = document.split("\n\n")

        for line in lines:
            words_ls = []
            tags_ls = []
            tags_with_start_stop = []
            for word_tag in line.split("\n"):
                word, tag = word_tag.split(" ")
                words_ls.append(word)
                tags_ls.append(tag)

            tag_ls_with_start_stop = ["START"] + tags_ls + ["STOP"]
            tags.append(tags_ls)
            tags_with_start_stop.append(tag_ls_with_start_stop)
            words.append(words_ls)
    return tags, tags_with_start_stop, words

def _preprocess_test_file(testing_file):
    test_words = []

    with open(testing_file,"r",encoding="utf8") as f:
        document = f.read().rstrip()
        lines = document.split("\n\n")

        for line in lines:
            word_ls = []
            for word in line.split("\n"):
                word_ls.append(word)
            test_words.append(word_ls)

    return test_words

In [91]:
def get_unique_elements(lst):
    return list(set(list(itertools.chain.from_iterable(lst))))

In [92]:
def get_unique_tags(tags):
    tags_unique = get_unique_elements(tags)
    tags_unique.sort()
    tags_unique_with_start_stop = ["START"] + tags_unique + ["STOP"]

    return tags_unique, tags_unique_with_start_stop


In [93]:
def get_transition_pairs(tags):
    transition_pairs = []

    for tag in tags:
        #yi-1 and yi tuples
        for tag1, tag2 in zip(tags[:-1], tags[1:]):
            transition_pairs.append([tag1, tag2])

    return transition_pairs

In [94]:
def count_y(tag, tags):
    tags_flattened = list(itertools.chain.from_iterable(tags))
    return tags_flattened.count(tag)

In [103]:
def get_transition_using_MLE(tags_unique_with_start_stop, transition_pairs,
                          tags_with_start_stop):
  
    transition = {}
    #create matrix dimensions
    for tag1 in tags_unique_with_start_stop[:-1]:
        transition_row = {}
        for tag2 in tags_unique_with_start_stop[1:]:
            transition_row[tag2] = 0.0
        transition[tag1] = transition_row

    # populate transition parameters with counts
    for tag1, tag2 in transition_pairs:
        transition[tag1][tag2] += 1

    # divide transition_count by count_yi, to get probability
    for tag1, transition_row in transition.items():
        count_yi = count_y(tag1, tags_with_start_stop)

        # words in training set
        for tag2, transition_count in transition_row.items():
            if count_yi == 0:
                transition[tag1][tag2] = 0.0
            else:
                transition[tag1][tag2] = transition_count / count_yi

    return transition

In [105]:
tags, tags_with_start_stop, words = _preprocess_training_file(es_train_data_file)
test_words = _preprocess_test_file(es_test_data_file)
tags_unique_with_start_stop, tags_unique_with_start_stop = get_unique_tags(
        tags)
transition_pairs = get_transition_pairs(tags_with_start_stop)
transition_matrix = get_transition_using_MLE(tags_unique_with_start_stop,
                                              transition_pairs,
                                              tags_with_start_stop)